In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from collections import Counter
import re

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
df = df.dropna(subset=['keyword', 'text', 'target'])

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [ ]:
df['cleaned_tweets'] = df['text'].apply(preprocess_text)
df['cleaned_keywords'] = df['keyword'].apply(lambda x: str(x).lower())

<ipython-input-10-1a8ba35c555c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweets'] = df['text'].apply(preprocess_text)
<ipython-input-10-1a8ba35c555c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_keywords'] = df['keyword'].apply(lambda x: str(x).lower())


In [ ]:
top_keywords = Counter(df['cleaned_keywords']).most_common(5)
top_keywords = [keyword for keyword, count in top_keywords]

In [ ]:
for keyword in top_keywords:
    df[f'keyword_{keyword}'] = df['cleaned_tweets'].apply(lambda x: 1 if keyword in x else 0)

<ipython-input-12-bdfbeaeb768c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'keyword_{keyword}'] = df['cleaned_tweets'].apply(lambda x: 1 if keyword in x else 0)
<ipython-input-12-bdfbeaeb768c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'keyword_{keyword}'] = df['cleaned_tweets'].apply(lambda x: 1 if keyword in x else 0)
<ipython-input-12-bdfbeaeb768c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['cleaned_tweets']).toarray()

In [ ]:
X = np.concatenate([X, df[[f'keyword_{kw}' for kw in top_keywords]].values], axis=1)

In [ ]:
df

,id,keyword,location,text,target,cleaned_tweets,cleaned_keywords,keyword_fatalities,keyword_armageddon,keyword_deluge,keyword_body%20bags,keyword_damage
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze http co lhyxeo...,ablaze,0,0,0,0,0
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt http co yao1e0...,ablaze,0,0,0,0,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking news nigeria flag set abl...,ablaze,0,0,0,0,0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying set ablaze,ablaze,0,0,0,0,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze http co q...,ablaze,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,jt_ruff23 cameronhacker wrecked,wrecked,0,0,0,0,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,three days work pretty much wrecked hahaha sho...,wrecked,0,0,0,0,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,fx forex trading cramer iger 3 words wrecked d...,wrecked,0,0,0,0,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,engineshed great atmosphere british lion gig t...,wrecked,0,0,0,0,0


In [ ]:
def get_keywords_from_tweet(tweet, n_keywords=5):
    tfidf_vectorizer = TfidfVectorizer(max_features=n_keywords)
    tfidf_matrix = tfidf_vectorizer.fit_transform([tweet])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray().flatten()
    word_score_pairs = list(zip(feature_names, tfidf_scores))
    sorted_words = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
    top_keywords = [word for word, score in sorted_words[:n_keywords]]

    return top_keywords

In [ ]:
df['top_keywords'] = df['cleaned_tweets'].apply
print(df[['text', 'top_keywords']])

                                                   text  \
31    @bbcmtd Wholesale Markets ablaze http://t.co/l...   
32    We always try to bring the heavy. #metal #RT h...   
33    #AFRICANBAZE: Breaking news:Nigeria flag set a...   
34                   Crying out for more! Set me ablaze   
35    On plus side LOOK AT THE SKY LAST NIGHT IT WAS...   
...                                                 ...   
7578   @jt_ruff23 @cameronhacker and I wrecked you both   
7579  Three days off from work and they've pretty mu...   
7580  #FX #forex #trading Cramer: Iger's 3 words tha...   
7581  @engineshed Great atmosphere at the British Li...   
7582  Cramer: Iger's 3 words that wrecked Disney's s...   

                                           top_keywords  
31    <bound method Series.apply of 31      bbcmtd w...  
32    <bound method Series.apply of 31      bbcmtd w...  
33    <bound method Series.apply of 31      bbcmtd w...  
34    <bound method Series.apply of 31      bbcmtd w...  
3

<ipython-input-18-b07603ed3b6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['top_keywords'] = df['cleaned_tweets'].apply


In [ ]:
df

,id,keyword,location,text,target,cleaned_tweets,cleaned_keywords,keyword_fatalities,keyword_armageddon,keyword_deluge,keyword_body%20bags,keyword_damage,top_keywords
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze http co lhyxeo...,ablaze,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt http co yao1e0...,ablaze,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking news nigeria flag set abl...,ablaze,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying set ablaze,ablaze,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze http co q...,ablaze,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,jt_ruff23 cameronhacker wrecked,wrecked,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,three days work pretty much wrecked hahaha sho...,wrecked,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,fx forex trading cramer iger 3 words wrecked d...,wrecked,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,engineshed great atmosphere british lion gig t...,wrecked,0,0,0,0,0,<bound method Series.apply of 31 bbcmtd w...


In [ ]:
def get_keywords_from_tweet(tweet, n_keywords=5):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([tweet])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray().flatten()
    word_score_pairs = list(zip(feature_names, tfidf_scores))
    sorted_words = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
    top_keywords = [word for word, score in sorted_words[:n_keywords]]

    return top_keywords
df['top_keywords'] = df['cleaned_tweets'].apply(lambda tweet: get_keywords_from_tweet(tweet, n_keywords=5))
print(df[['text', 'top_keywords']])

                                                   text  \
31    @bbcmtd Wholesale Markets ablaze http://t.co/l...   
32    We always try to bring the heavy. #metal #RT h...   
33    #AFRICANBAZE: Breaking news:Nigeria flag set a...   
34                   Crying out for more! Set me ablaze   
35    On plus side LOOK AT THE SKY LAST NIGHT IT WAS...   
...                                                 ...   
7578   @jt_ruff23 @cameronhacker and I wrecked you both   
7579  Three days off from work and they've pretty mu...   
7580  #FX #forex #trading Cramer: Iger's 3 words tha...   
7581  @engineshed Great atmosphere at the British Li...   
7582  Cramer: Iger's 3 words that wrecked Disney's s...   

                                          top_keywords  
31              [ablaze, bbcmtd, co, http, lhyxeohy6c]  
32                    [always, bring, co, heavy, http]  
33    [2nndbgwyei, aba, ablaze, africanbaze, breaking]  
34                               [ablaze, crying, set]  
35    

<ipython-input-20-793ce21360af>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['top_keywords'] = df['cleaned_tweets'].apply(lambda tweet: get_keywords_from_tweet(tweet, n_keywords=5))


In [ ]:
df

,id,keyword,location,text,target,cleaned_tweets,cleaned_keywords,keyword_fatalities,keyword_armageddon,keyword_deluge,keyword_body%20bags,keyword_damage,top_keywords
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze http co lhyxeo...,ablaze,0,0,0,0,0,"[ablaze, bbcmtd, co, http, lhyxeohy6c]"
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt http co yao1e0...,ablaze,0,0,0,0,0,"[always, bring, co, heavy, http]"
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking news nigeria flag set abl...,ablaze,0,0,0,0,0,"[2nndbgwyei, aba, ablaze, africanbaze, breaking]"
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying set ablaze,ablaze,0,0,0,0,0,"[ablaze, crying, set]"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze http co q...,ablaze,0,0,0,0,0,"[ablaze, co, http, last, look]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,jt_ruff23 cameronhacker wrecked,wrecked,0,0,0,0,0,"[cameronhacker, jt_ruff23, wrecked]"
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,three days work pretty much wrecked hahaha sho...,wrecked,0,0,0,0,0,"[days, family, hahaha, much, one]"
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,fx forex trading cramer iger 3 words wrecked d...,wrecked,0,0,0,0,0,"[7ennullkzm, co, cramer, disney, forex]"
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,engineshed great atmosphere british lion gig t...,wrecked,0,0,0,0,0,"[atmosphere, british, co, engineshed, gig]"


In [ ]:
df.to_csv('modified_tweets_dataset.csv', index=False)
from google.colab import files
files.download('modified_tweets_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
df1=pd.read_csv('/content/modified_tweets_dataset.csv')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import pad_sequence

df1['tokenized_text'] = df1['cleaned_tweets'].apply(lambda x: x.lower().split())

# Create vocabulary and add padding token
word_counts = Counter([word for sentence in df1['tokenized_text'] for word in sentence])
vocab = {word: i for i, (word, _) in enumerate(word_counts.items())}
vocab["<PAD>"] = len(vocab)  # Padding token
vocab_size = len(vocab)

# Function to create fixed-size context-target pairs
def create_context_target_pairs(tokenized_text, context_size, vocab):
    data = []
    for sentence in tokenized_text:
        for i, target_word in enumerate(sentence):
            context = [
                sentence[j] if 0 <= j < len(sentence) else "<PAD>"
                for j in range(i - context_size, i + context_size + 1) if j != i
            ]
            data.append((context, target_word))
    return data

# Create pairs with fixed-length contexts
context_size = 2  # Number of context words on each side
context_target_pairs = create_context_target_pairs(df1['tokenized_text'], context_size, vocab)

# Convert words to indices
def word_to_idx(word, vocab):
    return vocab.get(word, vocab["<PAD>"])

def context_to_idx(context, vocab):
    return [vocab[word] for word in context]

data = [
    (context_to_idx(context, vocab), word_to_idx(target, vocab))
    for context, target in context_target_pairs
]

# Dataset and custom collate_fn for padding
class CBOWDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        return torch.tensor(context, dtype=torch.long), torch.tensor(target, dtype=torch.long)

def collate_fn(batch):
    contexts, targets = zip(*batch)
    contexts = [torch.tensor(context, dtype=torch.long) for context in contexts]
    targets = torch.tensor(targets, dtype=torch.long)
    contexts_padded = pad_sequence(contexts, batch_first=True, padding_value=vocab["<PAD>"])
    return contexts_padded, targets

# Create DataLoader
batch_size = 32
dataset = CBOWDataset(data)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# CBOW Model
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context):
        embedded = self.embeddings(context)
        context_embedding = embedded.mean(dim=1)
        out = self.linear(context_embedding)
        return out

# Initialize and train the model
embedding_dim = 100
model = CBOWModel(vocab_size, embedding_dim)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for context, target in data_loader:
        optimizer.zero_grad()
        output = model(context)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")


<ipython-input-2-48eaf334d789>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  contexts = [torch.tensor(context, dtype=torch.long) for context in contexts]


Epoch 1, Loss: 22670.4827
Epoch 2, Loss: 18204.3260
Epoch 3, Loss: 15844.4781
Epoch 4, Loss: 13985.0737
Epoch 5, Loss: 12391.1931
Epoch 6, Loss: 10981.5580
Epoch 7, Loss: 9716.4692
Epoch 8, Loss: 8576.4018
Epoch 9, Loss: 7569.3184
Epoch 10, Loss: 6704.2422
